In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt 

import fastai
from fastai.tabular import *
from fastai.vision import *

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

torch.manual_seed(3)
torch.cuda.manual_seed(3)

%matplotlib inline

stim = np.load('../input/stim.npy')
df = pd.read_csv('../input/train.csv')
df.index = df.Id
df = df.fillna(df.mean())
df.Id = ['../data/%d.png' % i for i in range(50, 601)]

In [4]:
for i, image in enumerate(stim):
    if i < 50:
        continue
    else:
      Image.fromarray(image, mode='RGB').save("../data/%d.png" % i)  

In [19]:
cols = df.columns.values[1:].tolist()[:1]

In [25]:
data = ImageItemList.from_df(df, f'../data').random_split_by_pct() \
                    .label_from_df(cols=cols, label_cls=fastai.data_block.FloatList) \
                    .transform(get_transforms(), size=224).databunch()
data.normalize(imagenet_stats)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [26]:
data.train_dl.num_workers = 0
data.valid_dl.num_workers = 0
learn = create_cnn(data, models.resnet34)

learn.lr_find()
learn.recorder.plot()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.torch/models/resnet34-333f7ec4.pth
 87%|████████▋ | 76013568/87306240 [00:02<00:00, 27178378.37it/s]


KeyboardInterrupt: 

In [27]:
lr = 1e-3

In [ ]:
learn.fit_one_cycle(4, max_lr=lr)

In [ ]:
learn.save('learner0')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-6, 1e-3))

In [ ]:
learn.save('learner1')

In [ ]:
learn.fit_one_cycle(12, max_lr=slice(1e-6, 1e-3))

In [ ]:
learn.fit_one_cycle(12, max_lr=slice(1e-6, 1e-3))